# DMQC Report: WMO 4900883

## Current status and metadata: 

- APEX float inactive as of September 2011
- Deployed November 2006
- Completed 174 cycles in North Atlantic, appears to have been deployed off Southeast Grand Bank on NL AZMP
- In 'A' mode, gain of 0.9917 applied
- Calculated mean gain of 1.011

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from matplotlib.dates import datestr2num
import matplotlib.pyplot as plt

from netCDF4 import Dataset

from pathlib import Path
import numpy as np
import pandas as pd
import bgcArgoDMQC as bgc

wmo = 4900883 # insert WMO number here
fig_path = Path('../../../figures/') / f'{wmo}' # where to save figures
fig_path.mkdir(exist_ok=True)
bgc.io.get_argo(wmo, local_path=bgc.io.Path.ARGO_PATH, overwrite=False) # download the data to ensure you have up to date files
flt = bgc.sprof(wmo)
flt.clean()
gains = flt.calc_gains(ref='WOA') # calculate gain, can change to ref='NCEP' if in-air data is available
print(np.nanmean(gains))

## DOXY Audit Status

- Few points in the middle of the mission where there is some "spikiness" in the float surface saturation
- Cycles between 91-102 are flagged, but not all

In [ ]:
audit_file = list(Path('../../../audit/').glob('DOXY_WOA*'))[-1]
df = pd.read_csv(audit_file, sep='\t', header=25)
df = df.loc[df.WMO == wmo]
df['date'] = [datestr2num(t) for t in df['profile date']]
print(df[['cycle', 'profile date', 'WOA G_raw']])

## Visual QC

Check the profiles and any anomolous looking saturation values.

In [ ]:
g = flt.plot('gain', ref='WOA')
g.axes[0].set_title(f'WMO: {wmo}', loc='left', fontweight='bold')
g.axes[0].plot(df['date'], df['flt O2 %sat'], '*')
g.axes[1].plot(df['date'], df['WOA G_raw'], '*', zorder=3)
g.axes[0].plot(flt.df.SDN.loc[flt.df.PRES < 50], flt.df.O2Sat.loc[flt.df.PRES < 50], 'o', zorder=0, alpha=0.1)
fig_path.mkdir(exist_ok=True)
g.fig.savefig(fig_path / 'gain_initial.png', bbox_inches='tight', dpi=250)

Figure 1: Top panel: Float oxygen percent saturation (blue line) compared to WOA percent saturation (orange line), with DOXY audit flagged cycle percent saturation (green stars) and raw float percent saturation (orange circles). Bottom panel: calculated gains (blue dots) and flagged gains from DOXY audit (orange stars).

Notes: Lots of spiking in the profiles flagged by the audit, though not all of those profiles are flagged and some look reasonable. Outside of that mostly looks good. 

In [ ]:
flt.reset()
g = flt.plot('qcprofiles', varlist=['DOXY', 'DOXY_ADJUSTED', 'O2Sat', 'TEMP', 'PSAL'])
g.fig.savefig(fig_path / 'qcprofiles.png', bbox_inches='tight', dpi=250)
g = flt.plot('qcprofiles', varlist=['DOXY', 'DOXY_ADJUSTED', 'O2Sat', 'TEMP', 'PSAL'])
g.axes[0].set_ylim((200,0))
g.fig.savefig(fig_path / 'qcprofiles_shallow.png', bbox_inches='tight', dpi=250)

Figures 2,3: Profiles coloured by QC flag (1-2, green, 3, yellow, 4, red) on difference depth scales.

Notes: Can't really see the potentially problematic profiles based on the gain plot, though maybe a couple high points in the very surface. The points flagged as 3 shallower than 250 should probably be downgraded to 4. 

In [ ]:
bad_profiles = flt.df.loc[flt.df.CYCLE.isin(df['cycle'])]
bad_profiles

In [ ]:
fig, axes = plt.subplots(1, 4, sharey=True)
ax = axes[0]
flt.rm_fillvalue()
ax.plot(flt.df.O2Sat, flt.df.PRES, '.')
ax.plot(bad_profiles.O2Sat, bad_profiles.PRES, '.')
ax.set_ylabel('Pressure (dbar)')
ax.set_xlabel('Saturation %')

# ax.axvline(123.5)
# ax.axvline(118)
# ax.axvline(75)
# ax.axvline(80)
# ax.axvline(90)
# ax.axvline(45)
# ax.axvline(110)

# ax.axhline(12)
# ax.axhline(27)
# ax.axhline(42)

bad = (flt.O2Sat > 123.5)
bad = bad | ((flt.O2Sat < 80) & (flt.PRES < 40))
bad = bad | ((flt.O2Sat < 90) & (flt.PRES < 27))
bad = bad | ((flt.O2Sat < 42) & (flt.PRES > 200) & (flt.PRES < 300))
bad = bad | ((flt.O2Sat > 110) & (flt.PRES > 150) & (flt.PRES < 500))
bad = bad | ((flt.O2Sat < 65) & (flt.PRES > 1300))
bad = bad | ((flt.O2Sat > 95) & (flt.PRES > 1300))
print(sum(bad))
ax.plot(flt.df.O2Sat.loc[bad], flt.df.PRES.loc[bad], '^', color='red')

ax = axes[1]
flt.rm_fillvalue()
ax.plot(flt.df.DOXY, flt.df.PRES, '.')
ax.plot(bad_profiles.DOXY, bad_profiles.PRES, '.')
ax.set_ylim((250,0))
ax.set_xlabel('Diss. Oxygen ($\mathregular{\mu}$mol kg$^{-1}$)')

ax.plot(flt.df.DOXY.loc[bad], flt.df.PRES.loc[bad], '^', color='red')

ax = axes[2]
flt.rm_fillvalue()
ax.plot(flt.df.TEMP, flt.df.PRES, '.')
ax.plot(bad_profiles.TEMP, bad_profiles.PRES, '.')
ax.set_ylim((250,0))
ax.set_xlabel(f'Temperature ({chr(176)}C)')

ax.plot(flt.df.TEMP.loc[bad], flt.df.PRES.loc[bad], '^', color='red')

ax = axes[3]
flt.rm_fillvalue()
ax.plot(flt.df.PSAL, flt.df.PRES, '.')
ax.plot(bad_profiles.PSAL, bad_profiles.PRES, '.')
ax.set_ylim((250,0))
ax.set_xlabel(f'Salinity')

ax.plot(flt.df.PSAL.loc[bad], flt.df.PRES.loc[bad], '^', color='red')

fig.set_size_inches(1.8*fig.get_figwidth(), fig.get_figheight())
fig.savefig(fig_path / 'bad_pts_o2sat_doxy.png', bbox_inches='tight', dpi=250)



Figure 4: highlight potentially bad profiles, use horizontal/vertical lines to define "bad" indices. 

In [ ]:
# get the mean gain
flt.update_field('DOXY_QC', 3, where=flt.DOXY_QC == 1)
flt.update_field('DOXY_QC', 3, where=flt.DOXY_QC == 0)
flt.update_field('DOXY_ADJUSTED_QC', flt.DOXY_QC)
flt.update_field('DOXY_QC', 4, where=bad)
flt.update_field('DOXY_ADJUSTED_QC', 4, where=bad)
# make sure DOXY_QC is 3 instead of 0,1,2
flt.update_field('DOXY_QC', 3, where=flt.DOXY_QC.isin([0, 1, 2]))
# fill in DOXY_ADJUSTED_QC appropriately
flt.update_field('DOXY_ADJUSTED_QC', 4, where=flt.TEMP_ADJUSTED_QC == 4)
flt.update_field('DOXY_ADJUSTED_QC', 3, where=flt.PSAL_ADJUSTED_QC == 4)

flt.clean()
new_gains = flt.calc_gains(ref='WOA')

flt.update_field('DOXY_ADJUSTED', np.ma.masked_array(flt.gain*flt.DOXY, mask=flt.DOXY_ADJUSTED_QC.isin([4, 9])))
flt.update_field('DOXY_ADJUSTED_ERROR', bgc.calc_fixed_doxy_adjusted_error(flt.PSAL, flt.TEMP, flt.PRES))

# make sure the data is FillValues where adjusted values are still bad
flt.update_field('DOXY_ADJUSTED_QC', 4, where=flt.DOXY_ADJUSTED.isna())
flt.set_fillvalue('DOXY_ADJUSTED', where=flt.DOXY_ADJUSTED_QC.isin([4, 9]))
flt.set_fillvalue('DOXY_ADJUSTED_ERROR', where=flt.DOXY_ADJUSTED_QC.isin([4, 9]))

print(np.nanmean(new_gains))

In [ ]:
g = flt.plot('gain', ref='WOA')
g.axes[0].set_title(f'WMO: {wmo}', loc='left', fontweight='bold')
g.axes[0].plot(flt.df.SDN.loc[flt.df.PRES < 50], flt.df.O2Sat.loc[flt.df.PRES < 50], 'o', zorder=0, alpha=0.1)
# g.axes[0].plot(df['date'], df['flt O2 %sat'], '*')
# g.axes[1].plot(df['date'], df['WOA G_raw'], '*', zorder=3)
g.fig.savefig(fig_path / 'gain_final.png', bbox_inches='tight', dpi=250)

There is still a bit of spikiness in that area but I am hesitant to remove it as it looks well grouped with the rest of the profiles.

In [ ]:
flt.export_files(data_mode='D')

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
from netCDF4 import Dataset
import copy

def fill_nearest_neighbour(P, param):
    out = copy.deepcopy(param)
    ix = np.where(np.isnan(param[0,:]))
    for i in ix[0]:
        iy = np.where(P[0,:] - P[0,i] == np.nanmin(P[0,:] - P[0,i]))
        out[0,i] = param[0,iy[0][0]]

    return out

wmo = 4900883
loc = Path(f'/Users/GordonC/Documents/data/Argo/dac/meds/D/{wmo}/profiles/')
files = pd.read_csv(Path(f'../../../checker/summary/{wmo}/files.txt'))
varname = 'DOXY'
for fn in files.files:
    print(fn)
    nc = Dataset((loc / fn).absolute(), 'r+')
    pres = copy.deepcopy(nc['PRES'][:])
    param = copy.deepcopy(nc['DOXY_ADJUSTED_ERROR'][:])
    param = fill_nearest_neighbour(pres, param)
    nc['DOXY_ADJUSTED_ERROR'][:] = param
    nc.close()

In [ ]:
files = pd.read_csv(Path(f'../../../checker/summary/{wmo}/files.txt'))
varname = 'DOXY'
for fn in files.files:
    print(fn)
    nc = Dataset((loc / fn).absolute(), 'r+')
    error = copy.deepcopy(nc['DOXY_ADJUSTED_ERROR'][:])
    error.mask[0,2] = False
    print(error)
    error.data[0,2] = error.data[0,3]
    print(error)
    nc['DOXY_ADJUSTED_ERROR'][:] = error
    print(nc['DOXY_ADJUSTED_ERROR'][:] )
    nc.close()


In [ ]:
nc.close()